In [40]:
%load_ext autoreload
%autoreload all

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Programming

In [41]:
# TODO
#!{__import__('sys').executable} -m pip install --quiet 

## Importing `pyenergyplus`

In [42]:
# TODO ...

## `epems` - EnergyPlus API, Object-Oriented

<table style="text-align: center">
    <caption>Architecture Overview (*NIX)</caption>
    <tr>
        <th>EnergyPlus<br>OO Abstraction Layer</th>
        <th rowspan="2">&rlhar;</th>
        <th colspan="3">EnergyPlus<br>State Machine</th>
    </tr>
    <tr>
        <td>
            API<br>
            <code>epems</code>
        </td>
        <td>
            API<br>
            (<code>pyenergyplus</code>)<br>
            (<code>libenergyplusapi.so</code>)
        </td>
        <td>ABI<br>&rlhar;</td>
        <td>Kernel<br>(<code>energyplus</code>)</td>
    </tr>
</table>

### `epems` in Action

#### Setup

- (Context/State Management) Enter `epems` environment...
  Don't forget to `__close__` it when done \
  OR better yet, just use `python`'s `with`-statement:
  ```python
    import epems
    with epems.Environment() as e:
        # NOTE do whatever you need to do with `e`
        ...
  ```

In [43]:
import epems

# TODO !!!!!!!!!!
energyplus_path = epems.energyplus_path

In [44]:
env = epems.Environment().__enter__()

- 🎉

In [45]:
env._exec('--help')


PythonLinkage: Linked to Python Version: "3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:08:06) [GCC 11.3.0]"
Built on Platform: Ubuntu22.04_x86_64
Usage: energyplus [options] [input-file]
Options:
  -a, --annual                 Force annual simulation
  -c, --convert                Output IDF->epJSON or epJSON->IDF, dependent on
                               input file type
  -D, --design-day             Force design-day-only simulation
  -d, --output-directory ARG   Output directory path (default: current
                               directory)
  -h, --help                   Display help information
  -i, --idd ARG                Input data dictionary path (default: Energy+.idd
                               in executable directory)
  -j, --jobs ARG               Multi-thread with N threads; 1 thread with no
                               arg. (Currently only for G-Function generation)
  -m, --epmacro                Run EPMacro prior to simulation
  -p, --output-prefi

0

In [46]:
env = env.__enter__()

env._exec(
    # TODO
    '--design-day',
    '--output-directory', 'build/demo-eplus',
    '--weather', f'{energyplus_path}/WeatherData/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
    #f'{energyplus_path}/ExampleFiles/CoolingTower_VariableSpeed_MultiCell.idf'
    f'{energyplus_path}/ExampleFiles/ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
)

EnergyPlus Starting
EnergyPlus, Version 23.1.0-87ed9199d4, YMD=2023.09.16 22:02


Initializing Response Factors
Calculating CTFs for "INTERIORFURNISHINGS"
Calculating CTFs for "DROPCEILING"
Calculating CTFs for "INT_WALL"
Calculating CTFs for "EXT_SLAB_8IN_WITH_CARPET"
Calculating CTFs for "INT_SLAB_FLOOR"
Calculating CTFs for "NONRES_ROOF"
Calculating CTFs for "NONRES_EXT_WALL"
Calculating CTFs for "BASEMENT_WALL_EAST_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_SOUTH_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_SOUTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_WEST_CFACTOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing S

EnergyPlus Completed Successfully.


0

#### Variables

In [47]:
env.specs.variables

,variable_name,variable_key
0,Site Outdoor Air Drybulb Temperature,ENVIRONMENT
1,Site Outdoor Air Dewpoint Temperature,ENVIRONMENT
2,Site Outdoor Air Wetbulb Temperature,ENVIRONMENT
3,Site Wind Speed,ENVIRONMENT
4,Site Sky Temperature,ENVIRONMENT
...,...,...
2093,Zone Mechanical Ventilation No Load Heat Addit...,DATACENTER_BASEMENT_ZN_6
2094,Zone Mechanical Ventilation Heating Load Incre...,DATACENTER_BASEMENT_ZN_6
2095,Zone Mechanical Ventilation Heating Load Incre...,DATACENTER_BASEMENT_ZN_6
2096,Zone Mechanical Ventilation Heating Load Decre...,DATACENTER_BASEMENT_ZN_6


#### Actuators

In [48]:
env.specs.actuators

,component_type,control_type,actuator_key
0,Weather Data,Outdoor Dry Bulb,Environment;
1,Weather Data,Outdoor Dew Point,Environment;
2,Weather Data,Outdoor Relative Humidity,Environment;
3,Weather Data,Diffuse Solar,Environment;
4,Weather Data,Direct Solar,Environment;
...,...,...,...
5083,AirLoopHVAC,Availability Status,VAV_TOP WITH REHEAT;
5084,AirLoopHVAC,Availability Status,AIRLOOP DATACENTER BASEMENT;
5085,AirLoopHVAC,Availability Status,AIRLOOP DATACENTER BOT;
5086,AirLoopHVAC,Availability Status,AIRLOOP DATACENTER MID;


- Component types of the actuators...

In [49]:
env.specs.actuators['component_type'].unique()

array(['Weather Data', 'Schedule:Compact', 'Schedule:Constant',
       'Material', 'People', 'Lights', 'ElectricEquipment', 'Surface',
       'Zone', 'Zone Infiltration', 'Plant Loop Overall',
       'Supply Side Half Loop', 'Demand Side Half Loop',
       'Demand Side Branch', 'Plant Component Pipe:Adiabatic',
       'Plant Component WaterUse:Connections', 'Supply Side Branch',
       'Plant Component Pump:ConstantSpeed',
       'Plant Component WaterHeater:Mixed',
       'Plant Component Coil:Heating:Water',
       'Plant Component Pump:VariableSpeed',
       'Plant Component Boiler:HotWater',
       'Plant Component HeatExchanger:FluidToFluid',
       'Plant Component Chiller:Electric:ASHRAE205',
       'Plant Component Coil:Cooling:Water',
       'Plant Component Coil:Cooling:WaterToAirHeatPump:EquationFit',
       'Plant Component Coil:Heating:WaterToAirHeatPump:EquationFit',
       'Plant Component FluidCooler:TwoSpeed',
       'Plant Component HeaderedPumps:VariableSpeed',
     

In [50]:
env.specs.actuators[env.specs.actuators['component_type'] == 'Plant Component Chiller:Electric:ASHRAE205']

,component_type,control_type,actuator_key
1424,Plant Component Chiller:Electric:ASHRAE205,On/Off Supervisory,COOLSYS1 CHILLER1;
1427,Plant Component Chiller:Electric:ASHRAE205,On/Off Supervisory,COOLSYS1 CHILLER2;


In [51]:
env.specs.actuators[env.specs.actuators['component_type'] == 'Plant Component CoolingTower:VariableSpeed']

,component_type,control_type,actuator_key
1504,Plant Component CoolingTower:VariableSpeed,On/Off Supervisory,TOWERWATERSYS COOLTOWER 1;
1506,Plant Component CoolingTower:VariableSpeed,On/Off Supervisory,TOWERWATERSYS COOLTOWER 2;


In [52]:
env.specs.actuators[
    env.specs.actuators['component_type'].isin(
        ('Zone Temperature Control', 'Zone Humidity Control')
    )
]

,component_type,control_type,actuator_key
1631,Zone Temperature Control,Heating Setpoint,BASEMENT;
1632,Zone Temperature Control,Cooling Setpoint,BASEMENT;
1633,Zone Temperature Control,Heating Setpoint,CORE_BOTTOM;
1634,Zone Temperature Control,Cooling Setpoint,CORE_BOTTOM;
1635,Zone Temperature Control,Heating Setpoint,CORE_MID;
1636,Zone Temperature Control,Cooling Setpoint,CORE_MID;
1637,Zone Temperature Control,Heating Setpoint,CORE_TOP;
1638,Zone Temperature Control,Cooling Setpoint,CORE_TOP;
1639,Zone Temperature Control,Heating Setpoint,PERIMETER_BOT_ZN_3;
1640,Zone Temperature Control,Cooling Setpoint,PERIMETER_BOT_ZN_3;


#### Events

In [53]:
env.specs.events

,event_name
0,after_component_input
1,after_new_environment_warmup_complete
2,after_predictor_after_hvac_managers
3,after_predictor_before_hvac_managers
4,begin_new_environment
5,begin_system_timestep_before_predictor
6,begin_zone_timestep_after_init_heat_balance
7,begin_zone_timestep_before_init_heat_balance
8,begin_zone_timestep_before_set_current_weather
9,end_system_sizing


### TODO Simulation

In [54]:
env = env.__enter__()

# TODO !!!!! feature: read model without exec!

act = env.actuator(dict(
    component_type='Zone Temperature Control',
    control_type='Cooling Setpoint',
    actuator_key='CORE_MID'
))

class CallbackCounter:
    def __init__(self, fn=lambda *args, **kwargs: None):
        self._cnt = 0
        self._fn = fn

    def __call__(self, *args, **kwargs):
        self._cnt += 1
        return self._fn(*args, **kwargs)
    
    def summary(self):
        return f'{self!r} was called {self._cnt} times'
    
def demo_cb_ctrl():
    # TODO
    while not env._ready:
        #print('not ready!')
        return

    # TODO read variables here

    # TODO adjust actuator(s) accordingly
    act.value = 25.
    print(f'[act: {act.value}]', end=' ')

demo_cb = CallbackCounter(demo_cb_ctrl)

env.event(dict(event_name='after_component_input')).callback = demo_cb

env._exec(
    # TODO
    #'--design-day',
    #'--annual',
    '--output-directory', 'build/demo-eplus',
    '--weather', f'{energyplus_path}/WeatherData/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
    f'{energyplus_path}/ExampleFiles/ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
)

print(demo_cb.summary())

EnergyPlus Starting
EnergyPlus, Version 23.1.0-87ed9199d4, YMD=2023.09.16 22:03
Initializing Response Factors
Calculating CTFs for "INTERIORFURNISHINGS"
Calculating CTFs for "DROPCEILING"
Calculating CTFs for "INT_WALL"
Calculating CTFs for "EXT_SLAB_8IN_WITH_CARPET"
Calculating CTFs for "INT_SLAB_FLOOR"
Calculating CTFs for "NONRES_ROOF"
Calculating CTFs for "NONRES_EXT_WALL"
Calculating CTFs for "BASEMENT_WALL_EAST_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_SOUTH_CFACTOR"
Calculating CTFs for "BASEMENT_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_NORTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_SOUTH_CFACTOR"
Calculating CTFs for "DATACENTER_BASEMENT_ZN_6_WALL_WEST_CFACTOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combi

EnergyPlus Completed Successfully.


In [55]:
env.__exit__()